In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import pandas as pd
import numpy as np
import string, os 
import warnings
!pip install langdetect

warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)


  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=d1f95884dd889fafe2655402e5b5a1c93d7fc6f2a1c53ae4fd56f17ba92f95c0
  Stored in directory: c:\users\jgjoa\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


Artigo de Referência:

https://towardsdatascience.com/automatically-generate-hotel-descriptions-with-lstm-afa37002d4fc

https://github.com/susanli2016/NLP-with-Python/blob/master/Hotel%20Description%20Generation%20LSTM.ipynb


In [28]:
#hotel_df = pd.read_json(r"C:\Users\jtavares\Downloadsinventory_hotel_BR_pt.json")
hotel_df= pd.read_json(r'C:/Users/jgjoa/Desktop/FGV/inventory_hotel_BR_pt.json')
#all_descriptions = list(hotel_df['description'].desc.values)
#len(all_descriptions)

In [29]:
hotel_df = hotel_df[['name','description']]
hotel_df = hotel_df[0:500]
all_descriptions = list(hotel_df['description'].values)

hotel_df

,name,description
0,Best Western Hôtel New York Nice,"Located in Nice's city center, Best Western Ho..."
1,Comfort Inn Hyde Park,"In London's Bayswater district, the Comfort In..."
2,Best Western Plus Dayton Hotel & Suites,Breakfast is offered daily and rooms feature f...
3,"The Elizabeth Hotel, Autograph Collection","The Elizabeth Hotel, Autograph Collection feat..."
4,Grand Mercure Vila Olímpia,"Localizado no coração de São Paulo, o Grand Me..."
...,...,...
495,Kelta,"In the Hostería Kelta, located in the Province..."
496,WorldMark Running Y,"Offering an indoor swimming pool, this Klamath..."
497,Hotel Kirst,<h3>Localização Do Estabelecimento</h3>\nCom u...
498,Motel 6,A short drive from beautiful beaches and easil...


In [30]:
hotel_df.shape


(500, 2)

In [31]:
corpus = [x for x in all_descriptions]
corpus[:1]


["Located in Nice's city center, Best Western Hotel Lakmi Nice is located on Avenue Jean Medecin, Nice's main shopping street which leads to Place Massena. It is next to Nice Etoile Shopping Center and is just 1969 feet from Nice- Ville Train Station. Each spacious air-conditioned guest room has a flat- screen TV and a private bathroom fitted with a hairdryer. Some of the soundproofed rooms feature wrought iron furniture and some rooms offer a balcony. WiFi access is available throughout the hotel. A buffet breakfast is served every morning in the breakfast room at the Best Western Hotel Lakmi Nice. Newspapers are available for guests and there is also a bar on site. The Promenade des Anglais is a 13-minute walk from the hotel and Vieux Nice, the historic district, is a 15-minute walk away. Six private parking spaces are available on site at an additional cost, which cannot be booked in advance. ((* Sorry, this information is not available in the selected language and is shown in EN)."

In [32]:
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
t.fit_on_texts(corpus)

In [33]:
# A dictionary of words and their counts.
print(t.word_counts)

# A dictionary of words and how many documents each appeared in.
print(t.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(t.document_count)

# A dictionary of words and their uniquely assigned integers.
print(t.word_index)

OrderedDict([('located', 278), ('in', 1800), ("nice's", 2), ('city', 151), ('center', 403), ('best', 41), ('western', 34), ('hotel', 994), ('lakmi', 2), ('nice', 6), ('is', 2203), ('on', 285), ('avenue', 6), ('jean', 1), ('medecin', 1), ('main', 15), ('shopping', 73), ('street', 36), ('which', 64), ('leads', 1), ('to', 517), ('place', 44), ('massena', 1), ('it', 99), ('next', 10), ('etoile', 1), ('and', 2931), ('just', 119), ('1969', 4), ('feet', 70), ('from', 730), ('ville', 3), ('train', 27), ('station', 78), ('each', 130), ('spacious', 51), ('air', 135), ('conditioned', 79), ('guest', 113), ('room', 254), ('has', 137), ('a', 2585), ('flat', 148), ('screen', 151), ('tv', 220), ('private', 206), ('bathroom', 106), ('fitted', 25), ('with', 758), ('hairdryer', 51), ('some', 77), ('of', 853), ('the', 2940), ('soundproofed', 5), ('rooms', 505), ('feature', 75), ('wrought', 1), ('iron', 6), ('furniture', 12), ('offer', 63), ('balcony', 37), ('wifi', 109), ('access', 224), ('available', 718

In [34]:
print('Found %s unique tokens.' % len(t.word_index))

Found 7553 unique tokens.


In [21]:
# Tokenization
t = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokens(corpus):
    t.fit_on_texts(corpus)
    total_words = len(t.word_index) + 1
    
    input_sequences = []
    for line in corpus:
        token_list = t.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            
    return input_sequences, total_words

In [50]:
input_sequences, total_words = get_sequence_of_tokens(corpus)


In [52]:
input_sequences[:10]

[[45, 5],
 [45, 5, 2698],
 [45, 5, 2698, 94],
 [45, 5, 2698, 94, 33],
 [45, 5, 2698, 94, 33, 279],
 [45, 5, 2698, 94, 33, 279, 343],
 [45, 5, 2698, 94, 33, 279, 343, 7],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699, 1337],
 [45, 5, 2698, 94, 33, 279, 343, 7, 2699, 1337, 4]]

In [53]:
total_words

7554

In [54]:
# pad sequences 
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [55]:
def create_model(max_sequence_len, total_words):
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=max_sequence_len - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 727, 10)           75540     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 7554)              762954    
                                                                 
Total params: 882,894
Trainable params: 882,894
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [61]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [62]:
print(generate_text("hilton seattle downtown", 100, model, max_sequence_len))


Hilton Seattle Downtown Hotel Is Located In The Heart Of The Historic Of The Hotel In The Heart Of The Heart Of The Heart Of The Hotel The Hotel Is Within Walking Distance Of The Hotel And The Beach Of The Hotel The Hotel Is Within Walking Distance Of The Hotel And The Hotel Is Just Minutes Away From The Hotel And The Hotel Is A Short Drive Away The Hotel Is A Short Drive Away And A Variety Of The Hotel The Hotel Offers A Variety Of The Hotel In The Hotel In The Hotel In The Heart Of The Heart Of


In [64]:
print(generate_text("Holiday Inn Express - New York City, City-center hotel, walk to Madison Square Garden", 100, model, max_sequence_len))

Holiday Inn Express - New York City, City-Center Hotel, Walk To Madison Square Garden By Wyndham Wapakoneta Is A Short Drive From The Beach Of The Beach The Hotel Is Just A Short Drive From The Hotel And The Beach Of The City Of The Hotel The Hotel Is Just Close From The Historic Of The The Conditioned And The Hotel The Hotel Is Just Minutes From The Historic Of The Hotel The Hotel Is Within Walking Distance Of The Hotel The Hotel Is Just A Short Drive Away And A Variety Of The Hotel The Hotel Offers A Variety Of The Hotel In The Hotel In The Hotel In The Hotel In The


# Teste 2 - Apenas Hoteis Brasileiros (Descrição em português)

In [27]:
hotelBR_df = pd.read_json(r'C:/Users/jgjoa/Desktop/FGV/inventory_hotel_BR_pt.json')
#hotelBR_df = hotelBR_df[hotelBR_df['country']=='Brasil']

hotelBR_df  = hotelBR_df[hotelBR_df['short_description']!='']
hotelBR_df = hotelBR_df[0:1000]

In [28]:
from langdetect import detect
hotelBR_df = hotelBR_df[['name','short_description']]

hotelBR_df['idioma'] = ''
hotelBR_df.reset_index(inplace= True)

for i in range(len(hotelBR_df['short_description'])):
    hotelBR_df['idioma'][i] = detect(str(hotelBR_df['short_description'][i]))

hotelBR_df

,index,name,short_description,idioma
0,0,Best Western Hôtel New York Nice,"Nice, France. ((* Sorry, this information is n...",en
1,1,Comfort Inn Hyde Park,Notting Hill - Paddington Station Area. ((* So...,en
2,2,Best Western Plus Dayton Hotel & Suites,Há várias opções de lazer e entretenimento ao ...,pt
3,3,"The Elizabeth Hotel, Autograph Collection","Fort Collins North. ((* Sorry, this informatio...",en
4,4,Grand Mercure Vila Olímpia,"Localizado no coração de São Paulo, o Grand Me...",pt
...,...,...,...,...
995,997,Executive Royal Hotel West Edmonton,"West Edmonton. ((* Sorry, this information is ...",en
996,998,Courtyard Fort Lauderdale Coral Springs,"Coral Springs - Coconut Creek. ((* Sorry, this...",en
997,999,Hotel Palazzo Benci,Florence Central- Old Town - Duomo - Ponte Vec...,en
998,1000,Hampton Inn Vincennes,"Se procura lazer e entretenimento, poderá cont...",pt


In [26]:
#hotelBR_df

'pt'

In [29]:
hotelBR_df= hotelBR_df[hotelBR_df['idioma']=='pt']


In [30]:
hotelBR_df

,index,name,short_description,idioma
2,2,Best Western Plus Dayton Hotel & Suites,Há várias opções de lazer e entretenimento ao ...,pt
4,4,Grand Mercure Vila Olímpia,"Localizado no coração de São Paulo, o Grand Me...",pt
5,5,Bang Sarey Nordic Resort,"Alivie o stress com massagens, tratamentos cor...",pt
8,8,LandLord Hotel Semarang,Com uma estadia Candiland Apartment em Semaran...,pt
10,10,The Wine House Hotel Quinta da Pacheca,"Deleite-se com uma ida ao spa, que oferece mas...",pt
...,...,...,...,...
991,993,Super 8 by Wyndham Hardisty,Não perca as várias atividades recreativas e d...,pt
992,994,Hôtel Barrière de Lille,"Relaxa com massagens, tratamentos corporais e ...",pt
993,995,Holiday Inn Express & Suites - Oklahoma City A...,Experimente uma piscina interior e uma sala de...,pt
998,1000,Hampton Inn Vincennes,"Se procura lazer e entretenimento, poderá cont...",pt


In [32]:
all_descriptionsBR = list(hotelBR_df['short_description'].values)

hotelBR_df.describe()

,index
count,384.000000
mean,506.278646
std,292.383762
min,2.000000
25%,232.750000
50%,514.000000
75%,768.750000
max,1001.000000


In [33]:
hotelBR_df.columns

Index(['index', 'name', 'short_description', 'idioma'], dtype='object')

In [35]:
#hotelBR_df[['name','price_currency','possible_integrators','description','short_description','small_description']]

In [36]:
corpusBR = [x for x in all_descriptionsBR]
corpusBR[:1]


['Há várias opções de lazer e entretenimento ao seu dispor, incluindo uma piscina interior, uma banheira de hidromassagem e uma sala de fitness. As facilidades adicionais incluem Wi-fi grátis, armazenamento de esquis e uma loja de presentes/quiosque de']

In [38]:
tBR = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)
tBR.fit_on_texts(corpusBR)

In [39]:
# A dictionary of words and their counts.
print(tBR.word_counts)

# A dictionary of words and how many documents each appeared in.
print(tBR.word_docs)

# An integer count of the total number of documents that were used to fit the Tokenizer (i.e. total number of documents)
print(tBR.document_count)

# A dictionary of words and their uniquely assigned integers.
print(tBR.word_index)

OrderedDict([('há', 11), ('várias', 181), ('opções', 45), ('de', 876), ('lazer', 102), ('e', 765), ('entretenimento', 98), ('ao', 182), ('seu', 143), ('dispor', 143), ('incluindo', 189), ('uma', 478), ('piscina', 153), ('interior', 67), ('banheira', 42), ('hidromassagem', 46), ('sala', 117), ('fitness', 101), ('as', 160), ('facilidades', 72), ('adicionais', 86), ('incluem', 87), ('wi', 278), ('fi', 276), ('grátis', 259), ('armazenamento', 7), ('esquis', 7), ('loja', 32), ('presentes', 32), ('quiosque', 29), ('localizado', 14), ('no', 103), ('coração', 2), ('são', 39), ('paulo', 2), ('o', 164), ('grand', 2), ('mercure', 1), ('sp', 2), ('vila', 2), ('olimpia', 1), ('ex', 2), ('caesar', 1), ('business', 1), ('está', 30), ('próximo', 4), ('a', 239), ('grandes', 1), ('centros', 1), ('financeiros', 1), ('importantes', 1), ('empresas', 1), ('por', 16), ('perto', 1), ('também', 49), ('excelentes', 1), ('bares', 3), ('restaurantes', 4), ('shoppings', 1), ('variados', 2), ('locais', 5), ('entret

In [40]:
print('Found %s unique tokens.' % len(tBR.word_index))

Found 1113 unique tokens.


In [41]:
# Tokenization
tBR = Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

def get_sequence_of_tokensBR(corpusBR):
    tBR.fit_on_texts(corpusBR)
    total_wordsBR = len(tBR.word_index) + 1
    
    input_sequencesBR = []
    for line in corpusBR:
        token_list = tBR.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequencesBR.append(n_gram_sequence)
            
    return input_sequencesBR, total_wordsBR

In [42]:
input_sequencesBR, total_wordsBR = get_sequence_of_tokensBR(corpusBR)


In [43]:
input_sequencesBR[:10]

[[153, 12],
 [153, 12, 65],
 [153, 12, 65, 1],
 [153, 12, 65, 1, 28],
 [153, 12, 65, 1, 28, 2],
 [153, 12, 65, 1, 28, 2, 32],
 [153, 12, 65, 1, 28, 2, 32, 11],
 [153, 12, 65, 1, 28, 2, 32, 11, 17],
 [153, 12, 65, 1, 28, 2, 32, 11, 17, 18],
 [153, 12, 65, 1, 28, 2, 32, 11, 17, 18, 9]]

In [44]:
total_wordsBR

1114

In [45]:
# pad sequences 
def generate_padded_sequencesBR(input_sequencesBR):
    max_sequence_lenBR = max([len(x) for x in input_sequencesBR])
    input_sequencesBR = np.array(pad_sequences(input_sequencesBR, maxlen = max_sequence_lenBR, padding = 'pre'))
    predictorsBR, labelBR = input_sequencesBR[:,:-1],input_sequencesBR[:,-1]
    labelBR = ku.to_categorical(labelBR, num_classes = total_wordsBR)
    
    return predictorsBR, labelBR, max_sequence_lenBR

predictorsBR, labelBR, max_sequence_lenBR = generate_padded_sequencesBR(input_sequencesBR)

In [46]:
def create_modelBR(max_sequence_lenBR, total_wordsBR):
    modelBR = Sequential()
    
    # Add Input Embedding Layer
    modelBR.add(Embedding(total_wordsBR, 10, input_length=max_sequence_lenBR - 1))
    
    # Add Hidden Layer 1 - LSTM Layer
    modelBR.add(LSTM(100))
    modelBR.add(Dropout(0.1))
    
    # Add Output Layer
    modelBR.add(Dense(total_wordsBR, activation='softmax'))

    modelBR.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return modelBR

modelBR = create_modelBR(max_sequence_lenBR, total_wordsBR)
modelBR.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 55, 10)            11140     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1114)              112514    
                                                                 
Total params: 168,054
Trainable params: 168,054
Non-trainable params: 0
_________________________________________________________________


In [48]:
modelBR.fit(predictorsBR, labelBR, epochs=50, verbose=1)

Epoch 1/50
386/386 [==============================] - 21s 42ms/step - loss: 5.3071
Epoch 2/50
386/386 [==============================] - 10s 25ms/step - loss: 4.6266
Epoch 3/50
386/386 [==============================] - 11s 29ms/step - loss: 3.9561
Epoch 4/50
386/386 [==============================] - 15s 38ms/step - loss: 3.3992
Epoch 5/50
386/386 [==============================] - 21s 54ms/step - loss: 2.9787
Epoch 6/50
386/386 [==============================] - 21s 55ms/step - loss: 2.6547
Epoch 7/50
386/386 [==============================] - 21s 53ms/step - loss: 2.3988
Epoch 8/50
386/386 [==============================] - 21s 54ms/step - loss: 2.1856
Epoch 9/50
386/386 [==============================] - 21s 55ms/step - loss: 2.0177
Epoch 10/50
386/386 [==============================] - 21s 54ms/step - loss: 1.8658
Epoch 11/50
386/386 [==============================] - 21s 54ms/step - loss: 1.7378
Epoch 12/50
386/386 [==============================] - 22s 56ms/step - loss: 1.6259
E

In [49]:
def generate_textBR(seed_textBR, next_wordsBR, modelBR, max_seq_lenBR):
    for _ in range(next_wordsBR):
        token_listBR = tBR.texts_to_sequences([seed_textBR])[0]
        token_listBR = pad_sequences([token_listBR], maxlen=max_seq_lenBR-1, padding='pre')
        
        #predicted = model.predict_classes(token_list, verbose=0)
        predicted = np.argmax(modelBR.predict(token_listBR, verbose=0), axis=-1)
        output_word = ''
        
        for word,index in tBR.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_textBR = seed_textBR + " " + output_word
        
    return seed_textBR.title()

In [50]:
print(generate_textBR("Buzios Beach localizado em Buzios", 100, modelBR, max_sequence_lenBR))

Buzios Beach Localizado Em Buzios Atenas A Apenas 20 Metros Da Praia Não Temos Luxo Do Local Este Hotel Foi Totalmente Remodelado Em 36 Quartos À Internet Sem Fios Acessível O Alojamento O Hotel Oferece Ligação De Internet Por Wi Fi No Local Os Hóspedes Podem Contact Apreciarão A Receção Aberta 24 Horas Por Dia As Áreas Comuns São Adequadas Para Pessoas E N A Pontos De Artesanato Os Quartos E Os Aeroporto Os Quartos Os Hóspedes Podem Usufruir Da Ligação É Quartos Por Wi Fi Nas Áreas Para Bicicletas No Hotel Wi Fi Grátis Serviços De Negócio Todos Os Variados Locais De N Podem


In [51]:
print(generate_textBR("Hotel localizado em Copacabana", 100, modelBR, max_sequence_lenBR))

Hotel Localizado Em Copacabana Serviço De Hospedagem Simples Com Conforto E Ótima Localização Não Temos Luxo E De Lazer Em Nossa Infraestrutura Somos Uma Opção De Hospedagem Ideal Para Quem Quer Durante Os Dias Conhecer Os Passeios De Alojamento Wi Fi Nas Áreas Comuns Os Hóspedes Podem Contactar A Receção A Qualquer Hora Do Dia O Imóvel E A Receção A Qualquer Hora A Interesse Do Jardim Do Jardim E A Propriedade Oferece Oferece Receção 24 Horas E Os Local Os Hóspedes Podem Contact E A 7 5 Km Da Km Da Artesanato Fica A Cerca De Barreirinhas A 2 2 Km Da Casa
